In [71]:
from flask import Flask ,render_template,request
import os
from werkzeug.utils import secure_filename


app=Flask(__name__)
@app.route('/')
def index():
    return render_template('index.html')


@app.route('/train_model')
def train_model():
    return render_template('train_model.html')

In [72]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# Prototype for training function with Pandas DataFrame
def train_tensor(nbr_layers, nbr_neurals, activation, loss, optimizer, epoches, data, target_column, alpha=0.001):
    """
    nbr_layers: Number of layers (int)
    nbr_neurons: Number of neurons per layer (list of ints) 
    activation: Activation function per layer (list of strings, e.g., 'relu', 'sigmoid')
    alpha: Learning rate (float)
    loss: Loss function (string, e.g., 'mean_squared_error')
    optimizer: Optimizer function (string, e.g., 'adam')
    epochs: Number of epochs (int)
    data: Pandas DataFrame (data) containing features and target column
    target_column: Name of the target column for regression (string)
    """

    # Split the DataFrame into features (X) and target (y)
    X = data.drop(columns=[target_column])  # All columns except the target column
    y = data[target_column]  # Target column for prediction
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # Build the model
    model = models.Sequential()
    
    # Add input layer with the number of features (columns) in X
    model.add(layers.InputLayer(input_shape=(X_train.shape[1],)))
    
    # Add hidden layers
    for i in range(nbr_layers):
        model.add(layers.Dense(units=nbr_neurals[i], activation=activation[i]))
    
    # Add final layer for regression (output a single value)
    model.add(layers.Dense(1))
    
    # Compile the model with the given optimizer and learning rate
    optimizer_func = getattr(tf.keras.optimizers, optimizer)(learning_rate=alpha)
    model.compile(loss=loss, optimizer=optimizer_func)
    
    # Train the model
    history = model.fit(X_train, y_train, epochs=epoches, batch_size=32, validation_split=0.2)
    
    # Plot loss over epochs
    plt.figure(figsize=(12, 6))
    
    # Plotting loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()

    # Plotting accuracy (if applicable, for regression usually not used)
    if 'accuracy' in history.history:
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy over Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid()
    
    plt.savefig('static/images/model_performance_plot.png')  # Save the performance plot to a file

    # Evaluate the model on the test set
    test_loss = model.evaluate(X_test, y_test)

    # Predict and calculate accuracy manually (since it's a regression problem)
    y_pred = model.predict(X_test)
    
    # Calculate R^2 score (as a measure of accuracy for regression)
    r2 = r2_score(y_test, y_pred)
    
    print(f'Test Loss: {test_loss}')
    print(f'R^2 Score: {r2}')
    
    return model, history, test_loss, r2


In [73]:


# Specify the directory where uploaded files will be saved
UPLOAD_FOLDER = 'static/uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ensure the folder exists
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

# Route to train data and save the file
@app.route('/train_data', methods=["POST", "GET"])
def train_data():
    if request.method == "POST":
        # Prepare to collect neurons and activations
        neurons = []
        activations = []
        number_layers = int(request.form['number_layers'])

        for i in range(number_layers):
    # Get number of neurons for each layer (formatted with f-string)
            neurons.append(int(request.form.get(f'number_neurons_{i}')))
            
            # Get activation function for each layer (formatted with f-string)
            activations.append(request.form.get(f'activation_function_{i}'))

        print(neurons)
        # Get the value of 'epoches' from the form
        epoches = int(request.form['epoches'])
        # learning_rate = request.form['learning_rate']
        loss_function = request.form['loss_function']
        optimizer_function = request.form['optimizer_function']
        Target = request.form['Target']

        # Get the uploaded file
        file = request.files['file']
        file_name=file.filename
        # Ensure the file exists and is not empty
        # Secure the file name to avoid security issues
        filename = secure_filename(file.filename)
        # Save the file to the specified folder
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        data = pd.read_csv(file_path)
        # call the function of training 
        model, history,test_loss,r2 = train_tensor(number_layers, neurons, activations,loss_function, optimizer_function, epoches, data,Target)
    
            # Render a template and pass the number of layers and saved file info
    return render_template('test.html',summary=model.summary(),score=r2,test_loss=test_loss, activations=activations, neurons=neurons,epoches=epoches,loss_function=loss_function,optimizer_function=optimizer_function,Target=Target)


In [70]:
if __name__ == '__main__':
    app.run(debug=True,port=4000,use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:4000
Press CTRL+C to quit
127.0.0.1 - - [16/Oct/2024 23:15:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 23:15:40] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 23:15:40] "GET /static/css/responsive.css HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 23:15:40] "GET /static/images/item-background/item8.png HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 23:15:40] "GET /static/js/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 23:15:40] "GET /static/images/item-background/item7.png HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 23:15:40] "GET /static/images/item-background/item1.png HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 23:15:40] "GET /static/js/popper.min.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 23:15:40] "GET /static/icon/icon.css HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 23:15:40] "GET /static/css/animation.css HTTP/1.1" 304 -
127.0.0.1 - - [16/Oct/2024 23:15:40] "GET /static/css/shortcodes.css HTTP/1.1" 304 -
127.0.

[10, 5]


c:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/5


...

 Total params: 1,007 (3.94 KB)

 Trainable params: 335 (1.31 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 672 (2.63 KB)

127.0.0.1 - - [16/Oct/2024 23:17:49] "POST /train_data HTTP/1.1" 200 -
127.0.0.1 - - [16/Oct/2024 23:22:40] "GET /index.html HTTP/1.1" 404 -
